## Simulation in the Inventory Model


#### John Stachurski

This code simulates the inventory of a firm many times.

It generates a matrix with `num_paths` rows and `sim_length` columns.

One simulation of inventory dynamics corresponds to one row.  

The question is: how well does naive parallelization work?

In [1]:
import numpy as np
import numba

Parameters:

In [2]:
s, S = 10.0, 100.0
p = 0.4

Jitted but not parallel

In [3]:
@numba.jit(nopython=True)
def sim_paths_jitted(initial_x=50.0, 
                     num_paths=100000,
                     sim_length=400):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    # For each row
    for i in range(num_paths):
        
        dvals = np.random.geometric(p, size=sim_length) - 1
        
        # Populate the row
        for t in range(sim_length-1):
            x, d = X[i, t], dvals[t]
            if x <= s:
                y = max(S - d, 0)
            else:
                y = max(x - d, 0)
            X[i, t+1] = y

    return X


In [4]:
timeit X = sim_paths_jitted()

1.65 s ± 2.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
@numba.jit(nopython=True, parallel=True)
def sim_paths_jitted_parallel(initial_x=50.0, 
                             num_paths=100000,
                             sim_length=400):
    
    X = np.empty((num_paths, sim_length), dtype=np.int32)
    X[:, 0] = initial_x
    
    # For each row
    for i in numba.prange(num_paths):
        
        dvals = np.random.geometric(p, size=sim_length) - 1
        # Populate the row
        for t in range(sim_length-1):
            x, d = X[i, t], dvals[t]
            if x <= s:
                y = max(S - d, 0)
            else:
                y = max(x - d, 0)
            X[i, t+1] = y
            
    return X


In [7]:
timeit X = sim_paths_jitted_parallel()

235 ms ± 3.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
